1. 스타 데이터에서 유튜브 링크가 잘못된 개체들이 있음
2. 잘못된 유튜브 링크들을 올바른 링크로 업데이트하는 sql 쿼리문이 필요함 단,
 - 그룹 멤버인데 개인 채널이 없고 그룹 채널만 있는 경우? 그냥 넘어감
 - 공식 채널이 없는 경우? 넘어감
3. 잘못된 유튜브 링크 리스트들은 api(prod_오매핑 개체 리스트)에 접속해서 볼 수 있고, 검수 및 맵핑 후엔 해당 api에서 리스트들을 제거해야 함
4. 제거는 api(prod_오매핑 개체 checked 업데이트_body)에서 body에 검수한 series_id를 입력, send하면 제거됨
5. 제거 후, 작성한 sql 쿼리문을 종우님께 전달하면 끝!

### 잘못된 유튜브 주소 리스트 보기

[GET] https://kja5z8z5y4.execute-api.ap-northeast-2.amazonaws.com/prod/url_checker

x-api-key: cNLd2ubOiuavWXTCWYlV26PLDFuZ35I58Rkr99Hx
### 잘못된 유튜브 주소 삭제 (서비스에 노출되는 테이블 아님)

[DELETE] https://kja5z8z5y4.execute-api.ap-northeast-2.amazonaws.com/prod/url_checker

x-api-key: cNLd2ubOiuavWXTCWYlV26PLDFuZ35I58Rkr99Hx

- 삭제방법1: path param에 series_id 하나 추가 (...prod/url_checker/578)

- 삭제방법2: body에 series_id 리스트 추가
{
    "series_id": [787,100,1]
}

#### 잘못된 유튜브 채널 양식

https://www.youtube.com/watch?v=ladClnnJhqg&list=PL9K3xwFkFqWGvtbvCXqOJwCIGYgZMD_7-&index=12

#### 올바른 유튜브 채널 양식

https://www.youtube.com/channel/UC8HEl74jL3bLLwfDP1OALxw

- 대량 삭제가 필요할 시 저한테 요청해주세요

- 삭제할 양이 많지 않으면 donut관리자에서 직접 수정


### 검수한 리스트 제거
prod_오매핑 개체 checked 업데이트_body 에서 body에 검수한 series_id 입력

In [19]:
import requests as req
import pandas as pd
import time
import datetime

save_dir = '/Users/mycelebs/Desktop/데이터 팀 인턴 이지은/유튜브 매핑/sql/'

t = datetime.datetime.now()
now = t.strftime('%Y%m%d')

In [20]:
# api 접속해서 유튜브 url 오매핑된 데이터 로드
url = 'https://kja5z8z5y4.execute-api.ap-northeast-2.amazonaws.com/prod/url_checker'
res = req.get(url,headers={'x-api-key':'cNLd2ubOiuavWXTCWYlV26PLDFuZ35I58Rkr99Hx'})

j = json.loads(res.text)
j

[{'uye_pk': '518',
  'cd_name': '김용',
  'series_id': '18085557',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/kimyongz',
  'input_channel_id': 'https://www.youtube.com/kimyongz',
  'regist_date': '2021-05-09 10:00:47',
  'update_date': '2021-05-09 10:00:47',
  'donut_url': 'http://dev.mycelebs.com/donut/Celeb/ShowManageCeleb/18085557',
  'checked': 'False'},
 {'uye_pk': '519',
  'cd_name': '유빈',
  'series_id': '18085627',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/blingbling_official',
  'input_channel_id': 'https://www.youtube.com/blingbling_official',
  'regist_date': '2021-05-09 10:00:48',
  'update_date': '2021-05-09 10:00:48',
  'donut_url': 'http://dev.mycelebs.com/donut/Celeb/ShowManageCeleb/18085627',
  'checked': 'False'},
 {'uye_pk': '520',
  'cd_name': '반엘',
  'series_id': '18085636',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/playlist?list=PLCnJp7L8iutik4SZ8fqzlYym8sLtQ6D2k',
  'input_channel_id': 'https://www.youtube.com/pla

In [21]:
# 총 검수 갯수
len(j)

3

In [22]:
# 필요한 데이터만 가지고 와서,
bin_df = pd.DataFrame(j)

bin_df = bin_df.drop(['uye_pk', 'status', 'input_channel_id', 'regist_date', 'update_date', 'donut_url', 'checked'], axis=1)
bin_df

,cd_name,series_id,db_csu_url
0,김용,18085557,https://www.youtube.com/kimyongz
1,유빈,18085627,https://www.youtube.com/blingbling_official
2,반엘,18085636,https://www.youtube.com/playlist?list=PLCnJp7L...


In [24]:
# input에 올바른 url 입력
# 없으면 그냥 엔터

list_ = []
url = []

for idx, row in bin_df.iterrows():
    print(row['cd_name'], row['series_id'], row['db_csu_url'])
    url_input = input('url를 입력하세요')
    url.append(url_input)
    
    youtube = {'cd_name':row['cd_name'], 'series_id':row['series_id'], 'db_csu_url':row['db_csu_url'], 'new_url':url_input}
    list_.append(youtube)

김용 18085557 https://www.youtube.com/kimyongz
url를 입력하세요https://www.youtube.com/channel/UCqTzhbAsOFyP23FV8WuWEUA
유빈 18085627 https://www.youtube.com/blingbling_official
url를 입력하세요
반엘 18085636 https://www.youtube.com/playlist?list=PLCnJp7L8iutik4SZ8fqzlYym8sLtQ6D2k
url를 입력하세요https://www.youtube.com/channel/UCPcNTLQvLPjUxXzcsf92ywg


In [39]:
all_idx = []

for l in list_ :
    idx = l['series_id']
    all_idx.append(idx)

In [40]:
all_idx

['18085557', '18085627', '18085636']

In [25]:
# 맵핑한 url들을 업데이트 쿼리문으로 작성해주는 코드
# 저장된 sql 파일 종우님께 전달

df = pd.DataFrame(list_, columns = ['cd_name', 'series_id', 'db_csu_url', 'new_url'])
list_idx = []

for idx, row in df.iterrows():
    new_url = row['new_url']
    series_id = row['series_id']
    
    if row['new_url'] == '' :
        pass
    else :
        query = f'''UPDATE `celeb_site_url` SET `csu_url` = '{new_url}' `update_date` = curdate() where `cd_idx` = {series_id};\n'''
        print(query)
        list_idx.append(series_id)
        
        with open(save_dir + now + '_youtube.sql', 'at') as f:
            f.write(query)
    

UPDATE `celeb_site_url` SET `csu_url` = 'https://www.youtube.com/channel/UCqTzhbAsOFyP23FV8WuWEUA' `update_date` = curdate() where `cd_idx` = 18085557;

UPDATE `celeb_site_url` SET `csu_url` = 'https://www.youtube.com/channel/UCPcNTLQvLPjUxXzcsf92ywg' `update_date` = curdate() where `cd_idx` = 18085636;



In [41]:
# postman -> prod_오매핑 개체 checked 업데이트_body -> 검수한 리스트 제거하고
# sql 파일 첨부해서 보고

month = time.strftime('%m', time.localtime(time.time()))
day = time.strftime('%d', time.localtime(time.time()))

print('*{0}월 {1}일 스타 유튜브 검수 및 매핑 완료했습니다. @jjongwoo'.format(month, day))
print('- 검수 리스트 : {}명'.format(len(j)))
print(", ".join(all_idx), end='\n\n')
print('- 맵핑 리스트 : {}명'.format(len(list_idx)))
print(", ".join(list_idx), end='\n\n')

*05월 10일 스타 유튜브 검수 및 매핑 완료했습니다. @jjongwoo
- 검수 리스트 : 3명
18085557, 18085627, 18085636

- 맵핑 리스트 : 2명
18085557, 18085636



In [6]:
sort_j = sorted(j, key=lambda e: (int(e['series_id'])))
sort_j

[{'uye_pk': '509',
  'cd_name': '강승윤',
  'series_id': '35',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/OfficialYGWINNER',
  'input_channel_id': 'https://www.youtube.com/OfficialYGWINNER',
  'regist_date': '2021-04-14 10:04:24',
  'update_date': '2021-04-14 10:04:24',
  'donut_url': 'http://dev.mycelebs.com/donut/Celeb/ShowManageCeleb/35',
  'checked': 'False'},
 {'uye_pk': '510',
  'cd_name': '세븐',
  'series_id': '476',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/officialse7en',
  'input_channel_id': 'https://www.youtube.com/officialse7en',
  'regist_date': '2021-04-14 10:04:25',
  'update_date': '2021-04-14 10:04:25',
  'donut_url': 'http://dev.mycelebs.com/donut/Celeb/ShowManageCeleb/476',
  'checked': 'False'},
 {'uye_pk': '505',
  'cd_name': '육성재',
  'series_id': '687',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/officialbtob',
  'input_channel_id': 'https://www.youtube.com/officialbtob',
  'regist_date': '2021-04-14 10:04:24',
  'up

In [7]:
for i in range(len(sort_j)) :
    print(sort_j[i]['series_id'], sort_j[i]['db_csu_url'])

35 https://www.youtube.com/OfficialYGWINNER
476 https://www.youtube.com/officialse7en
687 https://www.youtube.com/officialbtob
1823 https://www.youtube.com/officialbtob
2347 https://www.youtube.com/officialbtob
2354 https://www.youtube.com/officialbtob
2355 https://www.youtube.com/officialbtob
2358 https://www.youtube.com/officialbtob
2433 https://www.youtube.com/user/AshGrayMH
284162 https://www.youtube.com/soundzssf
5313388 https://www.youtube.com/c/realgirlsproject
5313392 https://www.youtube.com/loonatheworld
5313400 https://www.youtube.com/c/PKENTERTAINMENT8
15595623 https://www.youtube.com/c/raonlee
17071673 https://www.youtube.com/JihoonyTV
17087000 https://www.youtube.com/MIRAEofficial
17287202 https://www.youtube.com/Jadeartistofficial
17287204 https://www.youtube.com/user/Tritopsofficial


In [34]:
## sequel pro 에서 cd_name 조회하기 위해
li = []
for i in range(len(j)) :
    idx = j[i]['series_id']
    li.append(idx)

print(", ".join(li), end='\n\n')

'''
select cd_name from `star_ko_data` where `series_id` in ( ~~~ )
'''

5313388, 5313392, 5313400, 2358, 1823, 687, 2355, 2354, 2347, 35, 476, 2433, 284162, 17071673, 17087000, 17287202, 17287204, 15595623



'\nselect cd_name from `star_ko_data` where `series_id` in ( ~~~ )\n'

In [29]:
df = pd.read_excel("/Users/mycelebs/Desktop/데이터 팀 인턴 이지은/유튜브 매핑/youtube______________.xlsx")
df

,Unnamed: 0,url,Unnamed: 2,seires_id,Unnamed: 4,name,Unnamed: 6
0,update `celeb_site_url` set `csu_url` = ',https://www.youtube.com/channel/UC0JHjcFtdI6Q2...,' `update_date` = curdate() where `cd_idx`=,5313388,;,NaN,update `celeb_site_url` set `csu_url` = 'https...
1,update `celeb_site_url` set `csu_url` = ',https://www.youtube.com/channel/UCOJplhB0wGQWv...,' `update_date` = curdate() where `cd_idx`=,5313392,NaN,NaN,update `celeb_site_url` set `csu_url` = 'https...
2,update `celeb_site_url` set `csu_url` = ',https://www.youtube.com/channel/UC5zgjY7c-A6Gw...,' `update_date` = curdate() where `cd_idx`=,5313400,NaN,NaN,update `celeb_site_url` set `csu_url` = 'https...
3,update `celeb_site_url` set `csu_url` = ',https://www.youtube.com/channel/UCuQA88xHVbW9u...,' `update_date` = curdate() where `cd_idx`=,476,NaN,NaN,update `celeb_site_url` set `csu_url` = 'https...
4,update `celeb_site_url` set `csu_url` = ',https://www.youtube.com/channel/UCBk1avPrqmTkz...,' `update_date` = curdate() where `cd_idx`=,2433,NaN,NaN,update `celeb_site_url` set `csu_url` = 'https...
5,update `celeb_site_url` set `csu_url` = ',https://www.youtube.com/channel/UCWUzOKDpiFLXu...,' `update_date` = curdate() where `cd_idx`=,17071673,NaN,NaN,update `celeb_site_url` set `csu_url` = 'https...
6,update `celeb_site_url` set `csu_url` = ',https://www.youtube.com/channel/UC0QBW1zC_WKWe...,' `update_date` = curdate() where `cd_idx`=,17087000,NaN,NaN,update `celeb_site_url` set `csu_url` = 'https...
7,update `celeb_site_url` set `csu_url` = ',https://www.youtube.com/channel/UC-rTjm_eUazJh...,' `update_date` = curdate() where `cd_idx`=,17287202,NaN,NaN,update `celeb_site_url` set `csu_url` = 'https...
8,update `celeb_site_url` set `csu_url` = ',https://www.youtube.com/channel/UCTd1iyNbO2VgZ...,' `update_date` = curdate() where `cd_idx`=,17287204,NaN,NaN,update `celeb_site_url` set `csu_url` = 'https...
9,update `celeb_site_url` set `csu_url` = ',https://www.youtube.com/channel/UCQn1FqrR2OCjS...,' `update_date` = curdate() where `cd_idx`=,15595623,NaN,NaN,update `celeb_site_url` set `csu_url` = 'https...


In [35]:
df['url']

0      https://www.youtube.com/channel/UC8HEl74jL3bLL...
1      https://www.youtube.com/channel/UCRpdlPk671uOM...
2      https://www.youtube.com/channel/UCYv4-BqKkgkMV...
3                                                    NaN
4      https://www.youtube.com/channel/UCo9TX7ZS3mMRt...
                             ...                        
438    https://www.youtube.com/channel/UC3mY2SKYhPjqI...
439    https://www.youtube.com/channel/UCg9bRZEGGrWWS...
440    https://www.youtube.com/channel/UC9Zdit-LdjPdX...
441                                                  NaN
442                                                  NaN
Name: url, Length: 443, dtype: object

In [36]:
df.shape

(443, 7)

In [37]:
null_data = df[df['url'].isnull()]
null_data

,Unnamed: 0,url,Unnamed: 2,seires_id,Unnamed: 4,name,Unnamed: 6
3,NaN,NaN,NaN,144,NaN,김수현,NaN
7,NaN,NaN,NaN,269,NaN,디오,NaN
9,NaN,NaN,NaN,350,NaN,박수진,NaN
12,NaN,NaN,NaN,440,NaN,산들,NaN
14,NaN,NaN,NaN,509,NaN,송민호,NaN
...,...,...,...,...,...,...,...
428,NaN,NaN,NaN,16494345,NaN,설호승,NaN
430,NaN,NaN,NaN,16600562,NaN,케이,NaN
431,NaN,NaN,NaN,16646074,NaN,아인,NaN
441,NaN,NaN,NaN,17615454,NaN,우철,NaN


In [60]:
240 + 203

443

In [38]:
notnull_data = df['url'].dropna(axis=0)
notnull_data

0      https://www.youtube.com/channel/UC8HEl74jL3bLL...
1      https://www.youtube.com/channel/UCRpdlPk671uOM...
2      https://www.youtube.com/channel/UCYv4-BqKkgkMV...
4      https://www.youtube.com/channel/UCo9TX7ZS3mMRt...
5      https://www.youtube.com/channel/UCJ_sOiMLKpEH3...
                             ...                        
436    https://www.youtube.com/channel/UC-rTjm_eUazJh...
437    https://www.youtube.com/channel/UC4UINNZHP5M2k...
438    https://www.youtube.com/channel/UC3mY2SKYhPjqI...
439    https://www.youtube.com/channel/UCg9bRZEGGrWWS...
440    https://www.youtube.com/channel/UC9Zdit-LdjPdX...
Name: url, Length: 240, dtype: object

In [39]:
idx = notnull_data.index

In [40]:
notnull_df = df.loc[idx, :]
notnull_df

,Unnamed: 0,url,Unnamed: 2,seires_id,Unnamed: 4,name,Unnamed: 6
0,update `celeb_site_url` set `csu_url` = ',https://www.youtube.com/channel/UC8HEl74jL3bLL...,' `update_date` = curdate() where `cd_idx`=,14,;,GOT7,update `celeb_site_url` set `csu_url` = 'https...
1,NaN,https://www.youtube.com/channel/UCRpdlPk671uOM...,NaN,47,NaN,개리,NaN
2,NaN,https://www.youtube.com/channel/UCYv4-BqKkgkMV...,NaN,82,NaN,글램,NaN
4,NaN,https://www.youtube.com/channel/UCo9TX7ZS3mMRt...,NaN,200,NaN,김태우,NaN
5,NaN,https://www.youtube.com/channel/UCJ_sOiMLKpEH3...,NaN,231,NaN,나인뮤지스,NaN
...,...,...,...,...,...,...,...
436,NaN,https://www.youtube.com/channel/UC-rTjm_eUazJh...,NaN,17287202,NaN,Jade,NaN
437,NaN,https://www.youtube.com/channel/UC4UINNZHP5M2k...,NaN,17378742,NaN,권영훈,NaN
438,NaN,https://www.youtube.com/channel/UC3mY2SKYhPjqI...,NaN,17416139,NaN,메이트리,NaN
439,NaN,https://www.youtube.com/channel/UCg9bRZEGGrWWS...,NaN,17509107,NaN,로맨티스코,NaN


In [41]:
notnull_df.to_excel("/Users/mycelebs/Desktop/notnull_youtube.xlsx")

In [ ]:
## 수정한 개체 delete

In [48]:
notnull_df['seires_id']

0            14
1            47
2            82
4           200
5           231
         ...   
436    17287202
437    17378742
438    17416139
439    17509107
440    17554658
Name: seires_id, Length: 240, dtype: int64

In [61]:
li = list(notnull_df['seires_id'])

In [56]:
for idx in li :
    url = 'https://kja5z8z5y4.execute-api.ap-northeast-2.amazonaws.com/prod/url_checker'
    res_ = req.delete(url + '/{}'.format(idx), headers={'x-api-key':'cNLd2ubOiuavWXTCWYlV26PLDFuZ35I58Rkr99Hx'})
    print(idx)

14
47
82
200
231
243
343
391
409
480
574
578
615
867
873
1004
1006
1016
1044
1070
1075
1087
1088
1098
1214
1223
1288
1365
1472
1901
2102
2198
2218
2219
2282
2283
2292
2296
2342
2345
2463
2464
2575
2621
3134
3235
3576
3580
163152
163155
163156
163157
284025
284119
284132
288276
288277
288288
288289
288305
290237
290703
292926
478576
478789
478919
563662
563995
572224
688693
697407
697468
697480
827854
827856
828028
1953279
2518440
2723843
3170965
3718088
3791345
4753422
5213896
5213900
5213905
5213909
5213914
5213941
5213942
5213943
5213955
5213960
5213966
5213976
5213982
5213983
5213986
5213987
5213989
5213990
5213992
5213993
5213995
5214004
5214005
5214011
5214015
5214019
5214021
5313388
5313392
5313404
5313415
5361666
5378200
5427762
5444292
5477394
5477395
5477396
5477398
5493968
5493970
5527287
5560429
5595575
5793582
5793583
6069160
6102162
6170942
6187404
6286064
6384650
6433954
6499766
6849293
6865722
6983041
6983046
7104509
7104511
7153203
7265729
7298166
7494143
7730356
846617

In [57]:
## 삭제됐는지 확인

url = 'https://kja5z8z5y4.execute-api.ap-northeast-2.amazonaws.com/prod/url_checker'
res = req.get(url,headers={'x-api-key':'cNLd2ubOiuavWXTCWYlV26PLDFuZ35I58Rkr99Hx'})

j = json.loads(res.text)
j

[{'uye_pk': '2',
  'cd_name': '???',
  'series_id': '571',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/playlist?list=PL78D049A0D5E0EBB3',
  'input_channel_id': 'PL78D049A0D5E0EBB3',
  'regist_date': '2021-04-03 10:00:58',
  'update_date': '2021-04-05 10:00:57'},
 {'uye_pk': '3',
  'cd_name': '???',
  'series_id': '669',
  'status': '400',
  'db_csu_url': 'http://www.youtube.com/playlist?list=PLXg9dfUntCmuNArgkJAD12FU4yFmPAdPj',
  'input_channel_id': 'PLXg9dfUntCmuNArgkJAD12FU4yFmPAdPj',
  'regist_date': '2021-04-03 10:01:01',
  'update_date': '2021-04-05 10:01:00'},
 {'uye_pk': '5',
  'cd_name': '??',
  'series_id': '533',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/playlist?list=PL15EBBC3899C5503A',
  'input_channel_id': 'PL15EBBC3899C5503A',
  'regist_date': '2021-04-03 10:01:14',
  'update_date': '2021-04-05 10:01:10'},
 {'uye_pk': '6',
  'cd_name': '??',
  'series_id': '995',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/playlist?list=P

In [58]:
len(j)

203

In [1]:
ll = '''
43
421
265
660
973
2762
290678
2758
2757
535
290679
290675
290674
290673
536
290677
12242025
798
2351
2357
2345
711
266
2338
2343
81
2457
481
1011
570
1100
1635
699
11628207
2542
446
646
922
321
2362
1009
2761
8
9
281114
11628241
284147
533
961
995
269
409
478
1540
2342
2346
616
437
276
13956996
2367
1578
2368
1920
2369
281122
284027
284028
10125105
10172445
3202
1929
3200
284029
3201
3199
3197
284031
290131
284033
284032
284030
284034
284035
284036
4310924
8053544
13896879
13896880
13896882
13896881
15322404
15946276
3196
15808059
15853570
15853568
15853569
15853567
290682
290620
290621
8397536
11904936
12013687
290680
2753
685
118
2754
1789
12396628
290676

'''

In [2]:
ll

'\n43\n421\n265\n660\n973\n2762\n290678\n2758\n2757\n535\n290679\n290675\n290674\n290673\n536\n290677\n12242025\n798\n2351\n2357\n2345\n711\n266\n2338\n2343\n81\n2457\n481\n1011\n570\n1100\n1635\n699\n11628207\n2542\n446\n646\n922\n321\n2362\n1009\n2761\n8\n9\n281114\n11628241\n284147\n533\n961\n995\n269\n409\n478\n1540\n2342\n2346\n616\n437\n276\n13956996\n2367\n1578\n2368\n1920\n2369\n281122\n284027\n284028\n10125105\n10172445\n3202\n1929\n3200\n284029\n3201\n3199\n3197\n284031\n290131\n284033\n284032\n284030\n284034\n284035\n284036\n4310924\n8053544\n13896879\n13896880\n13896882\n13896881\n15322404\n15946276\n3196\n15808059\n15853570\n15853568\n15853569\n15853567\n290682\n290620\n290621\n8397536\n11904936\n12013687\n290680\n2753\n685\n118\n2754\n1789\n12396628\n290676\n\n'

In [3]:
re_ll = ll.replace("\n", ", ")

In [4]:
re_ll

', 43, 421, 265, 660, 973, 2762, 290678, 2758, 2757, 535, 290679, 290675, 290674, 290673, 536, 290677, 12242025, 798, 2351, 2357, 2345, 711, 266, 2338, 2343, 81, 2457, 481, 1011, 570, 1100, 1635, 699, 11628207, 2542, 446, 646, 922, 321, 2362, 1009, 2761, 8, 9, 281114, 11628241, 284147, 533, 961, 995, 269, 409, 478, 1540, 2342, 2346, 616, 437, 276, 13956996, 2367, 1578, 2368, 1920, 2369, 281122, 284027, 284028, 10125105, 10172445, 3202, 1929, 3200, 284029, 3201, 3199, 3197, 284031, 290131, 284033, 284032, 284030, 284034, 284035, 284036, 4310924, 8053544, 13896879, 13896880, 13896882, 13896881, 15322404, 15946276, 3196, 15808059, 15853570, 15853568, 15853569, 15853567, 290682, 290620, 290621, 8397536, 11904936, 12013687, 290680, 2753, 685, 118, 2754, 1789, 12396628, 290676, , '